### Lấy dữ liệu toạ độ các quận, huyện, thành phố của Thành phố Hồ Chí Minh từ API

In [1]:
import requests
import time
import pandas as pd

def get_district_coordinates(city, country, districts):
    coordinates = {}
    # Nominatim URL để truy vấn tọa độ
    url = "https://nominatim.openstreetmap.org/search"
    
    # Lặp qua từng quận
    for district in districts:
        params = {
            "q": f"{district}, {city}, {country}",
            "format": "json",
            "addressdetails": 1,
            "limit": 1
        }
        
        try:
            headers = {
                 "User-Agent": "YourAppName/1.0 (21521882@gm.uit.edu.vn)"  # Thay thế bằng thông tin của bạn
            }

            response = requests.get(url, params=params, headers=headers)

            # Kiểm tra mã trạng thái phản hồi
            if response.status_code == 200:
                data = response.json()
                
                if data:
                    # Lấy tọa độ đầu tiên trong kết quả tìm kiếm
                    lat = data[0]["lat"]
                    lon = data[0]["lon"]
                    coordinates[district] = {"latitude": lat, "longitude": lon}
                    print(f"{district}: {coordinates[district]}")
                else:
                    print(f"Không tìm thấy tọa độ cho {district}")
            else:
                print(f"Lỗi HTTP {response.status_code} khi truy vấn {district}")
        except requests.exceptions.RequestException as e:
            print(f"Yêu cầu thất bại cho {district}: {e}")
        except ValueError:
            print("Phản hồi không ở định dạng JSON, nội dung:", response.text)
        
        # Thêm thời gian chờ để tuân thủ điều kiện API
        time.sleep(1)
    
    return coordinates

# Danh sách các quận ở TP Hồ Chí Minh
districts = [
    "District 1", "District 3", "District 4", "District 5", "District 6", 
    "District 7", "District 8", "District 10", "District 11", "District 12", 
    "Binh Tan District", "Binh Thanh District", "Go Vap District", "Phu Nhuan District", "Tan Binh District", 
    "Tan Phu District", "Thu Duc City", "Binh Chanh District", "Cu Chi District", "Can Gio District", 
    "Hoc Mon District", "Nha Be District"
]

# Gọi hàm để lấy tọa độ các quận ở TP Hồ Chí Minh, Việt Nam
coordinates = get_district_coordinates("Ho Chi Minh City", "Vietnam", districts)


# Lưu dữ liệu vào DataFrame
df = pd.DataFrame(coordinates)

# chuyển vị DataFrame
df = df.T
# chuyển index thành 1 cột
df = df.reset_index()

# đổi tên cột index thành Location
df = df.rename(columns={"index": "Location"})
# lưu dữ liệu toạ độ vào file CSV
df.to_csv("location_coordinates.csv", index=False)
df

District 1: {'latitude': '10.7773145', 'longitude': '106.6999907'}
District 3: {'latitude': '10.778639', 'longitude': '106.6870156'}
District 4: {'latitude': '10.7607328', 'longitude': '106.7075519'}
District 5: {'latitude': '10.7553616', 'longitude': '106.6685441'}
District 6: {'latitude': '10.745886', 'longitude': '106.6392921'}
District 7: {'latitude': '10.7375481', 'longitude': '106.7302238'}
District 8: {'latitude': '10.7217236', 'longitude': '106.6296151'}
District 10: {'latitude': '10.772732', 'longitude': '106.6683666'}
District 11: {'latitude': '10.7658124', 'longitude': '106.6474946'}
District 12: {'latitude': '10.8616036', 'longitude': '106.6609731'}
Binh Tan District: {'latitude': '10.7703708', 'longitude': '106.5996353'}
Binh Thanh District: {'latitude': '10.8017639', 'longitude': '106.7110322'}
Go Vap District: {'latitude': '10.8306171', 'longitude': '106.6779308'}
Phu Nhuan District: {'latitude': '10.7990972', 'longitude': '106.6802278'}
Tan Binh District: {'latitude': '

,Location,latitude,longitude
0,District 1,10.7773145,106.6999907
1,District 3,10.778639,106.6870156
2,District 4,10.7607328,106.7075519
3,District 5,10.7553616,106.6685441
4,District 6,10.745886,106.6392921
5,District 7,10.7375481,106.7302238
6,District 8,10.7217236,106.6296151
7,District 10,10.772732,106.6683666
8,District 11,10.7658124,106.6474946
9,District 12,10.8616036,106.6609731


### Lấy thông tin thời tiết của các quận, huyện, thành phố từ API

In [ ]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
import time
import numpy as np
# from datetime import datetime, timezone
# from pytz import timezone

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
def get_weather_data(district_params, time_range):
    url = "https://archive-api.open-meteo.com/v1/archive"

############################################################## Process hourly data: relative humidity, dew point, apparent temperature, pressure, cloud cover

    params = {
        "latitude": district_params["latitude"],
        "longitude": district_params["longitude"],
        "start_date": time_range["begin"],
        "end_date": time_range["end"],
        "hourly": ["relative_humidity_2m", "dew_point_2m", "apparent_temperature", "pressure_msl", "surface_pressure", "cloud_cover"],
        "timezone": "Asia/Bangkok"
    }
    responses = openmeteo.weather_api(url, params=params)

    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]

    # Process hourly data. The order of variables needs to be the same as requested.
    hourly = response.Hourly()
    hourly_relative_humidity_2m = hourly.Variables(0).ValuesAsNumpy()
    hourly_dew_point_2m = hourly.Variables(1).ValuesAsNumpy()
    hourly_apparent_temperature = hourly.Variables(2).ValuesAsNumpy()
    hourly_pressure_msl = hourly.Variables(3).ValuesAsNumpy()
    hourly_surface_pressure = hourly.Variables(4).ValuesAsNumpy()
    hourly_cloud_cover = hourly.Variables(5).ValuesAsNumpy()

    hourly_data = {"date": pd.date_range(
        start = pd.to_datetime(hourly.Time(), unit = "s", utc = True).date(),
        end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True).date(),
        freq = pd.Timedelta(seconds = hourly.Interval()),
        inclusive = "left"
    )}
    hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
    hourly_data["dew_point_2m"] = hourly_dew_point_2m
    hourly_data["apparent_temperature"] = hourly_apparent_temperature
    hourly_data["pressure_msl"] = hourly_pressure_msl
    hourly_data["surface_pressure"] = hourly_surface_pressure
    hourly_data["cloud_cover"] = hourly_cloud_cover

    df_hourly = pd.DataFrame(hourly_data)

    # Chuyển cột 'date' thành chỉ số thời gian
    df_hourly.set_index('date', inplace=True)

    # Tính toán dữ liệu hàng ngày
    df_hourly_grouped = df_hourly.resample('D').agg({
        'relative_humidity_2m': 'mean',  # Tính trung bình độ ẩm
        'dew_point_2m': 'mean',           # Tính trung bình điểm sương
        'pressure_msl': 'mean',           # Tính trung bình áp suất khí quyển
        'surface_pressure': 'mean',       # Tính trung bình áp suất bề mặt
        'cloud_cover': 'max'              # Lấy giá trị cao nhất của độ che phủ mây
    }).reset_index()


############################################################## Process daily data
    time.sleep(45)  # Wait to avoid hitting the rate limit
    params = {
        "latitude": district_params["latitude"],
        "longitude": district_params["longitude"],
        "start_date": time_range["begin"],
        "end_date": time_range["end"],
        "daily": ["weather_code", "temperature_2m_max", "temperature_2m_min", "temperature_2m_mean", "apparent_temperature_max", "apparent_temperature_min", "apparent_temperature_mean", "sunrise", "sunset", "daylight_duration", "sunshine_duration", "precipitation_sum", "rain_sum", "snowfall_sum", "precipitation_hours", "wind_speed_10m_max", "wind_gusts_10m_max", "wind_direction_10m_dominant", "shortwave_radiation_sum", "et0_fao_evapotranspiration"],
        "timezone": "Asia/Bangkok"
    }
    responses = openmeteo.weather_api(url, params=params)

    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]
    print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
    print(f"Elevation {response.Elevation()} m asl")
    print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
    print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

    # Process daily data. The order of variables needs to be the same as requested.
    daily = response.Daily()

    daily_weather_code = daily.Variables(0).ValuesAsNumpy()
    daily_temperature_2m_max = daily.Variables(1).ValuesAsNumpy()
    daily_temperature_2m_min = daily.Variables(2).ValuesAsNumpy()
    daily_temperature_2m_mean = daily.Variables(3).ValuesAsNumpy()
    daily_apparent_temperature_max = daily.Variables(4).ValuesAsNumpy()
    daily_apparent_temperature_min = daily.Variables(5).ValuesAsNumpy()
    daily_apparent_temperature_mean = daily.Variables(6).ValuesAsNumpy()
    daily_sunrise = daily.Variables(7).ValuesAsNumpy()
    daily_sunset = daily.Variables(8).ValuesAsNumpy()
    daily_daylight_duration = daily.Variables(9).ValuesAsNumpy()
    daily_sunshine_duration = daily.Variables(10).ValuesAsNumpy()
    daily_precipitation_sum = daily.Variables(11).ValuesAsNumpy()
    daily_rain_sum = daily.Variables(12).ValuesAsNumpy()
    daily_snowfall_sum = daily.Variables(13).ValuesAsNumpy()
    daily_precipitation_hours = daily.Variables(14).ValuesAsNumpy()
    daily_wind_speed_10m_max = daily.Variables(15).ValuesAsNumpy()
    daily_wind_gusts_10m_max = daily.Variables(16).ValuesAsNumpy()
    daily_wind_direction_10m_dominant = daily.Variables(17).ValuesAsNumpy()
    daily_shortwave_radiation_sum = daily.Variables(18).ValuesAsNumpy()
    daily_et0_fao_evapotranspiration = daily.Variables(19).ValuesAsNumpy()

    # print (daily_sunrise, daily_sunset)

    daily_data = {"date": pd.date_range(
        start = pd.to_datetime(daily.Time(), unit = "s", utc = True).date(),
        end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True).date(),
        freq = pd.Timedelta(seconds = daily.Interval()),
        inclusive = "left"
    )}

    daily_data["weather_code"] = daily_weather_code
    daily_data["temperature_2m_max"] = daily_temperature_2m_max
    daily_data["temperature_2m_min"] = daily_temperature_2m_min
    daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
    daily_data["apparent_temperature_max"] = daily_apparent_temperature_max
    daily_data["apparent_temperature_min"] = daily_apparent_temperature_min
    daily_data["apparent_temperature_mean"] = daily_apparent_temperature_mean
    daily_data["sunrise"] = daily_sunrise
    daily_data["sunset"] = daily_sunset
    daily_data["daylight_duration"] = daily_daylight_duration
    daily_data["sunshine_duration"] = daily_sunshine_duration
    daily_data["precipitation_sum"] = daily_precipitation_sum
    daily_data["rain_sum"] = daily_rain_sum
    daily_data["snowfall_sum"] = daily_snowfall_sum
    daily_data["precipitation_hours"] = daily_precipitation_hours
    daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max
    daily_data["wind_gusts_10m_max"] = daily_wind_gusts_10m_max
    daily_data["wind_direction_10m_dominant"] = daily_wind_direction_10m_dominant
    daily_data["shortwave_radiation_sum"] = daily_shortwave_radiation_sum
    daily_data["et0_fao_evapotranspiration"] = daily_et0_fao_evapotranspiration
   

    # Create a DataFrame from the daily data
    daily_dataframe = pd.DataFrame(data = daily_data)
    daily_dataframe = pd.merge(df_hourly_grouped , daily_dataframe, on='date', how='outer')


    # add district name, latitude, longitude to the dataframe
    daily_dataframe.insert(0, "location", district_params["location"])
    daily_dataframe.insert(1, "latitude", district_params["latitude"])
    daily_dataframe.insert(2, "longitude", district_params["longitude"])


    print("Đã lấy dữ liệu hàng ngày của ", district_params["location"] , "từ", daily_dataframe["date"].min(), "đến", daily_dataframe["date"].max())
    print("===============================================")
    return daily_dataframe

# Lấy dữ liệu thời tiết hàng ngày cho các quận ở TP Hồ Chí Minh từ 25/10/2014 đến 25/10/2024
# sẽ bị lùi 1 ngày
time_range = {
    "begin": "2014-10-26",
    "end": "2024-10-26"
}


############################################################## Lấy dữ liệu thời tiết hàng ngày cho 1 quận
# districts_params = {
#     "location": "District 1",
#     "latitude": "10.775659",
#     "longitude": "106.700424"
# }
# daily_weather_data = get_weather_data(districts_params, time_range)
# daily_weather_data


############################################################## Lấy dữ liệu thời tiết hàng ngày cho tất cả các quận ở TP Hồ Chí Minh
districts = pd.read_csv("location_coordinates.csv", header=0)
df = pd.DataFrame()
retry_attempts = 10  # Maximum retries
wait_time = 45  # Initial delay in seconds
step = 30  # Exponential backoff step

for index, district in districts.iterrows():
    district_params = {
        "location": district["Location"],
        "latitude": district["latitude"],
        "longitude": district["longitude"]
    }
    retry_count = 0

    while retry_count < retry_attempts:
        try:
            daily_weather_data = get_weather_data(district_params, time_range)
            # Append the data and break out of the retry loop on success
            df = pd.concat([df, daily_weather_data], ignore_index=True)
            time.sleep(wait_time)  # Wait to avoid hitting the rate limit

            break

        except Exception as e:
            # Check if this is a rate limit error
            print(f"Rate limit error at index {index}: {e}")
            retry_count += 1
            print(f"Retry attempt {retry_count} in {wait_time + retry_count * step} seconds")
            time.sleep(wait_time + retry_count * step)  # Exponential backoff

    if retry_count == retry_attempts:
        print(f"Failed to retrieve data for {index} after multiple attempts.")

df.to_csv("HCMCity_weather.csv", index=False)
print("Hoàn thành")